# Phase 1: Data Quality & Leakage Control
**Purpose:**  
- Understand dataset structure and metadata  
- Identify and remove data leakage columns  
- Perform full data cleaning (missing values, duplicates, type coercion, logical validation)  
- Output: `step1_quality_checked.csv`
## Step 1.1: Load & Inspect

In this notebook, we load the raw dataset, inspect its structure, types, and check for class imbalance.

In [12]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

# Add project root to path to import config
project_root = Path('../').resolve()
sys.path.append(str(project_root))

from config import RAW_DATA_PATH, LOGS_DIR, PROCESSED_DATA_DIR

# Display settings
pd.set_option('display.max_columns', None)

print(f'Loading data from: {RAW_DATA_PATH}')

Loading data from: E:\AI Trading Signal Prediction\AI-Trading-Signal-Prediction\data\raw\trades.csv


In [13]:
# Load Dataset with Robust Error Handling
try:
    df = pd.read_csv(RAW_DATA_PATH, low_memory=False)
except FileNotFoundError:
    raise FileNotFoundError(f"❌ File not found at {RAW_DATA_PATH}. Please ensure the data file exists.")

# Step 1.1: Inspect Shape and Types
print(f'Shape Data: {df.shape}')

# Column Types Summary
num_cols = df.select_dtypes(include=np.number).columns
cat_cols = df.select_dtypes(exclude=np.number).columns
print(f"\nNumeric Columns: {len(num_cols)} | Categorical Columns: {len(cat_cols)}")

print('\nData Types:')
print(df.dtypes)

display(df.head())

Shape Data: (76037, 60)

Numeric Columns: 52 | Categorical Columns: 8

Data Types:
id                           int64
coin                        object
price                      float64
status                       int64
TP5                        float64
TP7                        float64
TP9                        float64
TP10                       float64
TP12                       float64
TP14                       float64
TP16                       float64
TP18                       float64
TP20                       float64
TP25                       float64
TP30                       float64
TP35                       float64
TP40                       float64
TP45                       float64
TP50                       float64
TP1                          int64
low                        float64
high                       float64
open                       float64
close                      float64
volume                     float64
lastPrice                  float64
created

,id,coin,price,status,TP5,TP7,TP9,TP10,TP12,TP14,TP16,TP18,TP20,TP25,TP30,TP35,TP40,TP45,TP50,TP1,low,high,open,close,volume,lastPrice,created_at,closed_at,current_price,RSI,sl3,sl5,sl10,sl3_at,sl5_at,sl10_at,hit_first,hit_at,TP1_closed_at,price_1day_before_open,price_1day_before_low,price_1day_before_high,price_1day_before_close,price_3day_before_open,price_3day_before_low,price_3day_before_high,price_3day_before_close,1h,1day,3day,rsi_1d,rsi_3d,after_3days,ema21,ema50,ema100,ema21_all,ema50_all,ema100_all,atr_1h
0,1,ACA,0.180900,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0.178800,0.185100,0.184800,0.180900,2.907002e+06,0.186800,1710402299,1.710405e+09,0.061000,29.584352,1,1,1,1.710490e+09,1.710490e+09,1.710482e+09,1,1.710424e+09,1.710403e+09,0.189000,0.181000,0.238000,0.199800,0.145500,0.140500,0.238000,0.183200,"[[1710352800000,""0.20690000"",""0.20720000"",""0.1...","[[1709164800000,""0.11190000"",""0.11900000"",""0.1...","[[1706832000000,""0.08560000"",""0.09410000"",""0.0...",82.118708,81.202532,0.192400,0.192951,0.185194,0.162274,"[0.13297142857142857,0.13253766233766234,0.132...","[0.13616999999999999,0.1372064705882353,0.1383...",[0.162274],0.00848571
1,2,BONK,0.000034,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000034,0.000035,0.000035,0.000034,3.050473e+11,0.000035,1710402326,1.710406e+09,0.000017,64.361327,1,1,1,1.710425e+09,1.710490e+09,1.710475e+09,2,1.710425e+09,1.710403e+09,0.000029,0.000028,0.000033,0.000032,0.000031,0.000027,0.000033,0.000032,"[[1710352800000,""0.00003002"",""0.00003064"",""0.0...","[[1709164800000,""0.00001920"",""0.00002560"",""0.0...","[[1706745600000,""0.00001064"",""0.00001087"",""0.0...",61.595063,72.394425,0.000028,0.000032,0.000031,0.000031,"[3.131619047619047e-5,3.1180173160173154e-5,3....","[3.09396e-5,3.0927850980392155e-5,3.0893033294...",[3.0631099999999985e-5],0.00000165
2,3,BURGER,1.037500,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.007200,1.085000,1.058800,1.037500,1.855485e+06,1.022700,1710402328,1.710442e+09,0.426900,73.500545,1,1,1,1.710425e+09,1.710490e+09,1.710486e+09,2,1.710425e+09,1.710403e+09,0.894600,0.842800,0.912200,0.910600,0.848500,0.790000,1.129600,1.010700,"[[1710352800000,""0.86650000"",""0.87170000"",""0.8...","[[1709164800000,""0.67240000"",""0.72910000"",""0.6...","[[1706832000000,""0.44450000"",""0.46000000"",""0.4...",74.840909,87.792242,0.930100,0.950359,0.902997,0.856034,"[0.8112476190476191,0.8098887445887446,0.81037...","[0.8216839999999999,0.8227669803921567,0.82467...",[0.8560340000000001],0.06130714
3,4,CLV,0.124550,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1,0.120200,0.129700,0.127260,0.124550,8.503608e+06,0.129120,1710402332,1.710407e+09,0.040970,36.239316,1,1,1,1.710490e+09,1.710490e+09,1.710479e+09,1,1.710424e+09,1.710403e+09,0.112960,0.108930,0.149990,0.138560,0.096330,0.093840,0.150000,0.123880,"[[1710352800000,""0.14571000"",""0.14638000"",""0.1...","[[1709164800000,""0.06412000"",""0.06925000"",""0.0...","[[1706832000000,""0.04527000"",""0.04923000"",""0.0...",88.346939,92.553772,0.129550,0.132165,0.123169,0.104478,"[0.0815542857142857,0.08148025974025973,0.0813...","[0.08522419999999999,0.08585344705882351,0.086...",[0.10447800000000003],0.00841857
4,5,DIA,0.712600,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.702800,0.745200,0.738600,0.712600,7.079860e+05,0.730300,1710402343,1.710536e+09,0.342200,62.761506,1,1,1,1.710425e+09,1.710490e+09,1.710486e+09,2,1.710425e+09,1.710405e+09,0.655200,0.647200,0.712000,0.700100,0.599900,0.575500,0.717700,0.700100,"[[1710352800000,""0.68560000"",""0.69500000"",""0.6...","[[1709164800000,""0.54140000"",""0.55880000"",""0.5...","[[1706745600000,""0.38330000"",""0.49550000"",""0.3...",70.575000,83.219097,0.665100,0.699425,0.675592,0.641144,"[0.6013666666666667,0.5996515151515152,0.59827...","[0.607482,0.608918,0.610595725490196,0.6113370...",[0.641144],0.01987143


In [14]:
# Missing Values Check (Preview)
print('\nMissing Values (%):')
missing_stats = (df.isnull().mean() * 100).sort_values(ascending=False)
print(missing_stats.head(10))

print('\nMissing Values Statistics:')
print(missing_stats.describe())


Missing Values (%):
sl10_at          91.497560
lastPrice        89.343346
sl5_at           80.630483
TP50             80.431895
TP45             78.906322
TP40             76.969107
TP35             74.639978
current_price    72.410800
TP30             71.444165
TP25             66.996331
dtype: float64

Missing Values Statistics:
count    60.000000
mean     27.595644
std      31.015455
min       0.000000
25%       0.000000
50%      20.622855
75%      55.231992
max      91.497560
dtype: float64


## Step 1.2: Target Engineering & Leakage Audit
Since the dataset is raw, we need to derive the target variables (`target_hit`, `stop_hit`) and identify columns that contain future information (data leakage).

In [20]:
# =============================================================
# Step 1.2.1: Target Engineering + Timestamp Fix
# =============================================================

if 'hit_first' in df.columns:
    # --- Normalize hit_first values ---
    df["hit_first"] = pd.to_numeric(df["hit_first"], errors="coerce")

    # Remove trades with missing hit_first (incomplete or not labeled yet)
    before_rows = len(df)
    df = df[~df["hit_first"].isna()].copy().reset_index(drop=True)
    removed_nans = before_rows - len(df)
    if removed_nans > 0:
        pct_nan = removed_nans / before_rows * 100
        print(f"Removed {removed_nans} rows ({pct_nan:.2f}%) with NaN hit_first.")

    # Remove unresolved trades (class 2 = still open / unknown result)
    if df["hit_first"].isin([2]).any():
        unresolved = df[df["hit_first"] == 2]
        print(f"Removed {len(unresolved)} unresolved trades (hit_first == 2).")
        df = df[df["hit_first"] != 2].copy().reset_index(drop=True)

    # 1. Create target_hit based on hit_first == 1 (Confirmed Win)
    df["target_hit"] = (df["hit_first"] == 1).astype(int)

    # 2. Create stop_hit
    # Initialize to 0
    df["stop_hit"] = 0
    
    # Stop Condition: 
    # A. If target_hit is 1, stop_hit MUST be 0 (Target reached first).
    # B. If target_hit is 0, AND we have indication of stop (hit_first != 1) or SL data present.
    
    stop_loss_cols = ["sl3_at", "sl5_at", "sl10_at"]
    existing_sl_cols = [c for c in stop_loss_cols if c in df.columns]

    if existing_sl_cols:
        # Check if any SL timestamp exists
        sl_mask = df[existing_sl_cols].notna().any(axis=1)
        
        # Set stop_hit to 1 ONLY if NOT a target hit
        # This prevents false positives where SL data exists but TP was hit first.
        df.loc[(df["target_hit"] == 0) & sl_mask, "stop_hit"] = 1

    removed_pct = len(unresolved) / (len(df) + len(unresolved)) * 100
    print(f"📉 Removed {removed_pct:.2f}% of dataset as unresolved trades.")

    # --- Fix timestamps (created, hit, closed) ---
    for col in ["created_at", "hit_at", "closed_at"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Detect appropriate time unit
    time_unit = None
    if "created_at" in df.columns and not df["created_at"].dropna().empty:
        sample_val = df["created_at"].dropna().iloc[0]
        
        if sample_val > 1e12:
            time_unit = "ms"
        elif sample_val > 1e9:
            time_unit = "s"

    # Convert timestamps to datetime using detected unit
    if time_unit:
        for col in ["created_at", "hit_at", "closed_at"]:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], unit=time_unit, errors="coerce")

        # Compute time-to-event (in minutes)
        df["time_to_event"] = (
            (df["hit_at"].fillna(df["closed_at"]) - df["created_at"])
            .dt.total_seconds() / 60
        ).clip(lower=0)
        print("✅ Datetime columns fixed and 'time_to_event' calculated.")
    else:
        print("Could not detect timestamp scale — please verify raw values.")


    # --- Summary outputs ---
    # Assertion at the end
    assert not ((df["target_hit"] == 1) & (df["stop_hit"] == 1)).any(), \
        "Overlap between target_hit and stop_hit detected!"
    print("\n--- Targets Overview ---")
    print(f"# Unique target_hit classes: {df['target_hit'].nunique()}, "
          f"stop_hit: {df['stop_hit'].nunique()}")
    print(df[["target_hit", "stop_hit"]].head())
    
    if "time_to_event" in df.columns:
        print("\n--- Sample time_to_event ---")
        print(df[["created_at", "hit_at", "closed_at", "time_to_event"]].head())
        print("\n--- time_to_event Stats ---")
        print(df["time_to_event"].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95]).round(2))

else:
    print("Error: 'hit_first' column missing — cannot derive targets.")

Error: 'hit_first' column missing — cannot derive targets.


In [21]:
# Step 1.1: Check Class Imbalance (Recalculated on Engineered Targets)
def check_distribution(col, df):
    if col in df.columns:
        counts = df[col].value_counts(dropna=False)
        ratios = df[col].value_counts(normalize=True).mul(100).round(2)
        dist_df = pd.DataFrame({'Count': counts, 'Percentage': ratios})
        print(f'\n--- {col} Distribution ---')
        display(dist_df)
    else:
        print(f"Column {col} not found in dataset.")

check_distribution('target_hit', df)
check_distribution('stop_hit', df)

# Save target distribution report
if 'target_hit' in df.columns and 'stop_hit' in df.columns:
    dist_report_path = LOGS_DIR / "target_distribution_report.csv"
    if not dist_report_path.parent.exists():
        dist_report_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Saving counts of combinations for detailed consistency check
    df[["target_hit", "stop_hit"]].value_counts().to_csv(dist_report_path)
    print(f"\nSaved target distribution report to: {dist_report_path}")


--- target_hit Distribution ---


,Count,Percentage
target_hit,,
0,37105,70.46
1,15554,29.54



--- stop_hit Distribution ---


,Count,Percentage
stop_hit,,
0,51458,97.72
1,1201,2.28



Saved target distribution report to: E:\AI Trading Signal Prediction\AI-Trading-Signal-Prediction\logs\target_distribution_report.csv


In [22]:
# =============================================================
# Step 1.2b: Leakage Audit - Identify Future/Leak Columns
# =============================================================

leak_features = [
    "hit_first", "hit_at", "TP1_closed_at",
    "sl3_at", "sl5_at", "sl10_at",
    "closed_at", "after_3days", "current_price"
]

present_leakers = [col for col in leak_features if col in df.columns]

print("\nPotential Leakage Columns Found:")
for col in present_leakers:
    print(f" - {col}")

print(f"Total potential leak columns: {len(present_leakers)}")

# Save leakage report
leak_report_path = LOGS_DIR / "leakage_features_report.txt"

if not leak_report_path.parent.exists():
    leak_report_path.parent.mkdir(parents=True, exist_ok=True)

with open(leak_report_path, "w") as f:
    f.write("\n".join(present_leakers))
print(f"\n✅ Leakage columns report saved to {leak_report_path}")


Potential Leakage Columns Found:
Total potential leak columns: 0

✅ Leakage columns report saved to E:\AI Trading Signal Prediction\AI-Trading-Signal-Prediction\logs\leakage_features_report.txt


In [23]:
# =============================================================
# Step 1.2c: Drop Confirmed Leaks
# =============================================================

print("\nDropping confirmed leakage columns...")
cols_to_drop = [col for col in leak_features if col in df.columns]
df_clean = df.drop(columns=cols_to_drop, errors='ignore')
print(f"Dropped {len(cols_to_drop)} columns. New shape: {df_clean.shape}")

# Verify
remaining = [c for c in leak_features if c in df_clean.columns]
if remaining:
    print(f"Warning: Leaks remain: {remaining}")
else:
    print("✅ All confirmed leaks removed.")

# Update main df for saving
df = df_clean


Dropping confirmed leakage columns...
Dropped 0 columns. New shape: (52659, 54)
✅ All confirmed leaks removed.


In [24]:
# Sample Save (for logging)
sample_path = LOGS_DIR / 'sample_preview.csv'
if not sample_path.parent.exists():
    sample_path.parent.mkdir(parents=True, exist_ok=True)

if not df.empty:
    df.sample(min(100, len(df))).to_csv(sample_path, index=False)
    print(f"\nSaved sample preview to: {sample_path}")


Saved sample preview to: E:\AI Trading Signal Prediction\AI-Trading-Signal-Prediction\logs\sample_preview.csv


In [25]:
# Save Baseline with Targets (Step 1 Output)

# Use PROCESSED_DATA_DIR from config
baseline_path = PROCESSED_DATA_DIR / "step1_targets_created.csv"

if not baseline_path.parent.exists():
    baseline_path.parent.mkdir(parents=True, exist_ok=True)

df.to_csv(baseline_path, index=False)
print(f"\n✅ Clean dataset with targets saved to: {baseline_path}")


✅ Clean dataset with targets saved to: E:\AI Trading Signal Prediction\AI-Trading-Signal-Prediction\data\processed\step1_targets_created.csv


In [26]:
# =============================================================
# Step 1.3: Missing Values Handling
# =============================================================

print("\n--- Missing Values Handling ---")

# Recalculate missing percentages on the current (cleaned) df
missing_perc = df.isna().mean() * 100
missing_perc = missing_perc.sort_values(ascending=False)

# 1️⃣ Identify high-missing columns
HIGH_MISSING_THRESH = 80
high_missing_cols = missing_perc[missing_perc > HIGH_MISSING_THRESH].index.tolist()

print(f"Columns with >{HIGH_MISSING_THRESH}% missing: {high_missing_cols}")

# 2️⃣ Drop columns with excessive missingness
if high_missing_cols:
    df_cleaned = df.drop(columns=high_missing_cols, errors='ignore')
    print(f"Dropped {len(high_missing_cols)} columns with very high missing ratios.")
else:
    df_cleaned = df.copy()

# 3️⃣ Basic imputation for numeric and categorical columns
for col in df_cleaned.columns:
    if df_cleaned[col].isna().any():
        if df_cleaned[col].dtype in [float, int]:
            median_val = df_cleaned[col].median()
            df_cleaned[col] = df_cleaned[col].fillna(median_val)
        else:
            mode_val = df_cleaned[col].mode()[0] if not df_cleaned[col].mode().empty else "Unknown"
            df_cleaned[col] = df_cleaned[col].fillna(mode_val)

# 4️⃣ Confirm cleanup
remaining_missing = df_cleaned.isna().mean().max()
print(f"✅ All missing values handled. Max remaining NaNs: {remaining_missing:.2f}%")

# 5️⃣ Save cleaned dataset
step1_clean_final_path = PROCESSED_DATA_DIR / "step1_clean_final.csv"
df_cleaned.to_csv(step1_clean_final_path, index=False)
print(f"✅ Final cleaned dataset saved to: {step1_clean_final_path}")

# Optional: Print summary of removed columns for the log
if high_missing_cols:
    print("\nDropped columns summary:")
    for c in high_missing_cols:
        print(f" - {c}")


--- Missing Values Handling ---
Columns with >80% missing: ['lastPrice', 'TP50', 'TP45']
Dropped 3 columns with very high missing ratios.
✅ All missing values handled. Max remaining NaNs: 0.00%
✅ Final cleaned dataset saved to: E:\AI Trading Signal Prediction\AI-Trading-Signal-Prediction\data\processed\step1_clean_final.csv

Dropped columns summary:
 - lastPrice
 - TP50
 - TP45


In [27]:
# =============================================================
# Step 1.4: Data Integrity & Consistency Check
# =============================================================

print("\n--- Step 1.4: Data Integrity & Consistency Check ---")

# 1️. Duplicate Check (exact row duplicates)
dup_count = df_cleaned.duplicated().sum()
print(f"🔍 Found {dup_count} duplicate rows.")

if dup_count > 0:
    df_cleaned = df_cleaned.drop_duplicates()
    print(f"✅ Removed {dup_count} duplicate rows. New shape: {df_cleaned.shape}")
else:
    print("✅ No duplicate rows found.")

# 2️. Check for duplicated IDs (if 'id' column exists)
if "id" in df_cleaned.columns:
    id_dupes = df_cleaned["id"].duplicated().sum()
    if id_dupes > 0:
        print(f"⚠️ Found {id_dupes} duplicate IDs, investigate before modeling.")
    else:
        print("✅ No duplicate IDs found.")

# 3️. Data Type Sanity Check
print("\n--- Column Type Summary ---")
print(df_cleaned.dtypes.value_counts())

# If any object columns supposed to be numeric, convert them safely
object_cols = df_cleaned.select_dtypes(include='object').columns.tolist()
if object_cols:
    for col in object_cols:
        try:
            df_cleaned[col] = pd.to_numeric(df_cleaned[col])
        except (ValueError, TypeError):
            continue
    print(f"Checked potential numeric conversions for {len(object_cols)} object columns.")
    converted_cols = [
    col for col in object_cols
    if pd.api.types.is_numeric_dtype(df_cleaned[col])
    ]
    print(f" Successfully converted {len(converted_cols)} columns to numeric: {converted_cols}")

# 4. Basic Outlier Check on numeric columns
num_cols = df_cleaned.select_dtypes(include=np.number).columns
outlier_stats = {}

for col in num_cols:
    q1, q3 = df_cleaned[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    outlier_ratio = ((df_cleaned[col] < lower) | (df_cleaned[col] > upper)).mean() * 100
    outlier_stats[col] = outlier_ratio

# Print top 5 columns with highest outlier ratio
outlier_report = pd.Series(outlier_stats).sort_values(ascending=False).head(10).round(2)
print("\n--- Top 5 Numeric Columns by Outlier Ratio (% of rows) ---")
print(outlier_report)

# 5.Sanity checks for OHLC
mask_invalid = (
    (df_cleaned['high'] < df_cleaned[['open','close','low']].max(axis=1)) |
    (df_cleaned['low'] > df_cleaned[['open','close','high']].min(axis=1)) |
    (df_cleaned['close'] <= 0) | (df_cleaned['open'] <= 0)
)
if mask_invalid.any():
    print(f"⚠️ Removing {mask_invalid.sum()} rows with invalid OHLC structure.")
    df_cleaned = df_cleaned[~mask_invalid]

# 6. Save post-quality dataset
quality_checked_path = PROCESSED_DATA_DIR / "step1_quality_checked.csv"
df_cleaned.to_csv(quality_checked_path, index=False)

print(f"\n✅ Quality-checked dataset saved to: {quality_checked_path}")


--- Step 1.4: Data Integrity & Consistency Check ---
🔍 Found 0 duplicate rows.
✅ No duplicate rows found.
✅ No duplicate IDs found.

--- Column Type Summary ---
float64           34
int64              8
object             8
datetime64[ns]     1
Name: count, dtype: int64
Checked potential numeric conversions for 8 object columns.
 Successfully converted 0 columns to numeric: []

--- Top 5 Numeric Columns by Outlier Ratio (% of rows) ---
rsi_3d                     23.81
sl3                        23.08
price_3day_before_open     22.75
price_1day_before_open     22.75
price_1day_before_high     22.75
price_1day_before_low      22.75
price_3day_before_high     22.75
price_3day_before_low      22.75
price_3day_before_close    22.75
price_1day_before_close    22.75
dtype: float64

✅ Quality-checked dataset saved to: E:\AI Trading Signal Prediction\AI-Trading-Signal-Prediction\data\processed\step1_quality_checked.csv
